In [2]:
# Отдельная генерация и сохранение train/text массивов для xgbooster  (2017-07-02)

In [3]:
import sys,os,datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as skm
from __future__ import print_function

In [4]:
print(np.__version__);
print(pd.__version__);

1.12.1
0.19.2


In [5]:
import  cv2 as cv
cv.__version__

'3.2.0'

In [6]:
sys.path.append('../Python')
from helper import formFH, paths_input

In [7]:
binsData = 96

trLabels,trDirTIF,trDirJPG,teDirTIF,teDirJPG = paths_input()
if False : #True :
    trDirI = trDirJPG
    teDirI = teDirJPG
    ttExt  = '.jpg'
else :
    trDirI = trDirTIF
    teDirI = teDirTIF
    ttExt  = '.tif'
trWork, teWork = '../Work/Train', '../Work/Test'
print (trDirI,teDirI, trWork, teWork)
print (trLabels,trDirTIF,trDirJPG,teDirTIF,teDirJPG)

../Data/train-tif-v2 ../Data/test-tif-v2 ../Work/Train ../Work/Test
../Data/train_v2.csv ../Data/train-tif-v2 ../Data/train-jpg ../Data/test-tif-v2 ../Data/test-jpg-v2


In [8]:
xx=os.listdir(trDirI); xx.sort(); xx[:6]

['train_0.tif',
 'train_1.tif',
 'train_10.tif',
 'train_100.tif',
 'train_1000.tif',
 'train_10000.tif']

In [9]:
labels_df = pd.read_csv(trLabels)
labels_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [10]:
# Build list with unique labels
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [11]:
# Add features (0 or 1) for every label as dataframe field 
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
#labels_df.head()

In [12]:
weather_labels = ['clear', 'partly_cloudy', 'haze', 'cloudy']
land_labels = ['primary', 'agriculture', 'water', 'cultivation', 'habitation' ]
rare_labels = [l for l in label_list if labels_df[label_list].sum()[l] < 2000]
#rare_labels              

In [13]:
labels = label_list; #weather_labels;
nameList =labels_df[labels_df[labels].sum(axis=1)>0].image_name.tolist(); len(nameList)
labelList=labels_df[labels_df[labels].sum(axis=1)>0][labels].as_matrix();
#labelList[:6,:]
#labels_df.head()

In [73]:
def formImExt (nf, resize=(32,32), printOK=False) :
    nx = None
    try : 
        ni = cv.imread(nf,-1); 
        if (ni is not None) :
            if not ((ni.shape[2]==3) or (ni.shape[2]==4)) and printOK : print('----- error ---- shape:',ni.shape,nf)
            if (ni.shape[2]==3) :
                nx = cv.resize(ni,resize)
            if (ni.shape[2]==4) :
                #r,g,b,n = ni[:,:,2],ni[:,:,1],ni[:,:,0],ni[:,:,3]
                r,g,b,n = cv.resize(ni[:,:,2],resize),cv.resize(ni[:,:,1],resize),cv.resize(ni[:,:,0],resize),cv.resize(ni[:,:,3],resize)
                dv,dw   = np.divide((r-n),(r+n+0.01)), np.divide((g-n),(g+n+0.01))
                nx      = np.array([r,g,b,n,dv,dw]).T; 
    except BaseException as e :
        print(nf,e); nx = None;
    
    if nx is None and printOK : 
        print('------ None:',nf); nx = None
        
    return(nx)

def formImHist (nf, count, printOK=False) :
    
    def hist1 (nf1,bins) :
        h,_ = np.histogram(nf1.ravel(),bins=bins)
        return h.tolist()
    
    def histN (nf,bins) :
        h = []
        for i in range(0,nf.shape[2]) :
            h = h + hist1(nf[:,:,i],bins=bins);
        return h
             
    def histNX (nf,bins) :
        h = []
        for i in range(0,nf.shape[2]) :
            h = h + hist1(nf[:,:,i],bins=bins[i]);
        return h
    
    def calculateBins (low,high,count) :
        size    = float(high-low)/float(count)
        bins = [low+x*size for x in range(count+1)]; #print(count,size,len(bins),bins[:3],bins[-3:])
        return (bins)
             
    nx = None
    try : 
        ni = cv.imread(nf,-1); #print(ni.shape)
        if (ni is not None) :
            
            if printOK : print(nf,ni.shape)

            if not ((ni.shape[2]==3) or (ni.shape[2]==4)) and printOK : 
                print('----- error ---- shape:',ni.shape,nf);
            if  ni.shape[2]==3 :
                bins = calculateBins(0,255,count)
                nx = histN(ni,bins)
            if  ni.shape[2]==4 : 
                bins0 = calculateBins(0,65535,count)
                bins1 = calculateBins(-16,16,count)
                bins  = [bins0,bins0,bins0,bins0,bins1,bins1]
                r,g,b,n = ni[:,:,2], ni[:,:,1], ni[:,:,0], ni[:,:,3]
                dv,dw   = np.divide((r-n),(r+n+0.01)), np.divide((g-n),(g+n+0.01))
                ni      = np.array([r,g,b,n,dv,dw]).T; 
                nx = histNX(ni,bins)
            
    except BaseException as e :
        print(nf,e); nx = None;
    
    if nx is None and printOK : 
        print('------ None:',nf); nx = None
        
    return(nx)

In [114]:
def generateArrays( dirIn, nameAsk, binsData, debugOK=False) :
    #nameAsk = os.listdir(dirIn); print(len(nameAsk))
    trOX, trOY, i, size = [], [], 0, len(nameAsk)
    if debugOK : size = 3
    print(datetime.datetime.now(),len(nameAsk),size)
    for nn in nameAsk[0:size] :
        nf = os.path.join(dirIn,nn);
        nx = formImHist(nf,binsData,printOK=debugOK)
        if (nx is not None) :
            trOX.append(nx)
            trOY.append(nn)
        i += 1
        if (i%5000==0) : print(datetime.datetime.now(),"\t",i,"\t",nn)
            
    trOX = np.array(trOX);
    trOY = np.array([os.path.splitext(x)[0] for x in trOY]);

    print(datetime.datetime.now())
    return (trOX, trOY)

def generateTest ( dirIn, binsData, debugOK=False) :
    nameAsk = os.listdir(dirIn); #print(len(nameAsk))
    return(generateArrays(dirIn,nameAsk,binsData,debugOK=debugOK))

def generateTrain( dirIn, binsData, nameList, debugOK=False) :
    ##nameAsk = os.listdir(dirIn); #print(len(nameAsk))
    return(generateArrays(dirIn,nameList,binsData,debugOK=debugOK))


In [141]:
# Генерация для JPG
binsList = [96,128,256]
ttExt    = '.jpg'
debug = True

In [150]:
dirIn    = trDirJPG
trOY = labelList;
for bds in binsList :
    print ('-------->: bins={} Ext={} input dir={}'.format(bds,ttExt,dirIn))
    nameListExt = [x+ttExt for x in nameList]
    trOX, _     = generateTrain(dirIn,bds,nameListExt,debugOK=debug)
    np.save('../Work/t-'+str(10000+bds)+ttExt+'-XX-train.npy',trOX)
    np.save('../Work/t-'+str(10000+bds)+ttExt+'-YY-train.npy',trOY)
    #print (bds,trOX.shape,trOY.shape, (trOX.shape[1]/bds)*bds, trOX.shape[1]/3)dirIn    = teDirJPG

-------->: bins=96 Ext=.jpg input dir=../Data/train-jpg
2017-07-03 22:40:52.920080 40479 3
../Data/train-jpg/train_0.jpg (256, 256, 3)
../Data/train-jpg/train_1.jpg (256, 256, 3)
../Data/train-jpg/train_2.jpg (256, 256, 3)
2017-07-03 22:40:52.952614
-------->: bins=128 Ext=.jpg input dir=../Data/train-jpg
2017-07-03 22:40:52.979654 40479 3
../Data/train-jpg/train_0.jpg (256, 256, 3)
../Data/train-jpg/train_1.jpg (256, 256, 3)
../Data/train-jpg/train_2.jpg (256, 256, 3)
2017-07-03 22:40:53.001773
-------->: bins=256 Ext=.jpg input dir=../Data/train-jpg
2017-07-03 22:40:53.027506 40479 3
../Data/train-jpg/train_0.jpg (256, 256, 3)
../Data/train-jpg/train_1.jpg (256, 256, 3)
../Data/train-jpg/train_2.jpg (256, 256, 3)
2017-07-03 22:40:53.049705


In [149]:
dirIn    = teDirJPG
for bds in binsList :
    print ('-------->: bins={} Ext={} input dir={}'.format(bds,ttExt,dirIn))
    trOX, trOY = generateTest(dirIn,bds,debugOK=debug)
    np.save('../Work/t-'+str(10000+bds)+ttExt+'-XX-test.npy',trOX)
    np.save('../Work/t-'+str(10000+bds)+ttExt+'-YY-test.npy',trOY)
    #print (bds,trOX.shape,trOY.shape, (trOX.shape[1]/bds)*bds, trOX.shape[1]/3)

-------->: bins=96 Ext=.jpg input dir=../Data/test-jpg-v2
2017-07-03 22:40:10.042580 61191 3
../Data/test-jpg-v2/test_10002.jpg (256, 256, 3)
../Data/test-jpg-v2/test_10003.jpg (256, 256, 3)
../Data/test-jpg-v2/test_10004.jpg (256, 256, 3)
2017-07-03 22:40:10.065418
-------->: bins=128 Ext=.jpg input dir=../Data/test-jpg-v2
2017-07-03 22:40:10.094771 61191 3
../Data/test-jpg-v2/test_10002.jpg (256, 256, 3)
../Data/test-jpg-v2/test_10003.jpg (256, 256, 3)
../Data/test-jpg-v2/test_10004.jpg (256, 256, 3)
2017-07-03 22:40:10.115285
-------->: bins=256 Ext=.jpg input dir=../Data/test-jpg-v2
2017-07-03 22:40:10.143617 61191 3
../Data/test-jpg-v2/test_10002.jpg (256, 256, 3)
../Data/test-jpg-v2/test_10003.jpg (256, 256, 3)
../Data/test-jpg-v2/test_10004.jpg (256, 256, 3)
2017-07-03 22:40:10.164356


In [153]:
##--------------------- Examples save/load procedure

In [154]:
if False :
    trX = np.load('../Work/train-data-XX'+str(binsData)+ttExt+'.npy')
    trY = np.load('../Work/train-data-YY'+str(binsData)+ttExt+'.npy')
    print(trX.shape,trY.shape)

In [155]:
if False :
    np.save('../Work/test-data-XX'+str(binsData)+ttExt+'.npy',trOX)
    np.save('../Work/test-data-YY'+str(binsData)+ttExt+'.npy',trOY)